In [82]:
import pandas as pd
!ls ..data/raw/cartchar1000.csv
df = pd.read_csv('../data/raw/cartchar1000.csv')

ls: ..data/raw/cartchar1000.csv: No such file or directory


In [83]:
df.head()

,Unnamed: 0,object,scale,angle_deg,scattered_re_0,scattered_im_0,scattered_re_1,scattered_im_1,scattered_re_2,scattered_im_2,...,scattered_re_45,scattered_im_45,scattered_re_46,scattered_im_46,scattered_re_47,scattered_im_47,scattered_re_48,scattered_im_48,scattered_re_49,scattered_im_49
0,0,bone,420.88090,19.364860,-1.011516,0.678834,0.035420,0.265856,0.192819,-0.051677,...,-0.267690,0.202817,-0.164190,-0.192668,-0.173832,-0.182212,-0.319378,0.025581,-0.842201,0.371946
1,1,bone,441.56550,4.486947,-0.750103,0.400853,-0.306706,0.050576,0.316519,0.111739,...,0.013590,0.376455,-0.038690,0.147007,-0.223715,0.159206,-0.426532,0.518053,-0.650314,0.670647
2,2,bone,433.78740,-11.966660,-0.744530,0.355113,-0.217149,0.045633,0.109288,0.010012,...,0.136397,0.228325,-0.243060,0.050679,0.070367,0.048502,0.072496,0.303502,-0.867145,0.585742
3,3,hook,381.64570,27.261385,-1.250636,0.519960,-0.119485,0.453141,0.197689,0.112159,...,-0.888447,-0.339273,-0.443849,0.131576,-0.452241,-0.315346,0.835928,1.124528,-1.913356,0.944437
4,4,treasure,389.77283,29.188704,-0.879843,0.201698,0.309552,0.614370,-0.064069,-0.308829,...,0.688197,0.450446,-0.303251,0.343404,0.062388,-0.350611,0.273985,0.471243,-0.949470,0.382002


In [84]:
# create a neural network 
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

X = df[df.filter(regex=("scattered*")).columns].values
y_categories = df['object']
y_unique = y_categories.unique()
num_categories = len(y_unique)
y = np.zeros(y_categories.shape[0], np.int32)
for i in range(num_categories):
    name = y_unique[i]
    y[y_categories == name] = i
    
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=567)
    
model = keras.Sequential()
model.add(keras.layers.Dense(X.shape[1], activation='sigmoid'))
model.add(keras.layers.Dense(8, activation='sigmoid'))
model.add(keras.layers.Dense(8, activation='sigmoid'))
model.add(keras.layers.Dense(num_categories, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 426us/sample - loss: 1.1303 - acc: 0.3486
Epoch 2/100
700/700 [==============================] - 0s 85us/sample - loss: 1.0997 - acc: 0.3486
Epoch 3/100
700/700 [==============================] - 0s 79us/sample - loss: 1.0910 - acc: 0.3486
Epoch 4/100
700/700 [==============================] - 0s 75us/sample - loss: 1.0868 - acc: 0.3486
Epoch 5/100
700/700 [==============================] - 0s 76us/sample - loss: 1.0814 - acc: 0.3700
Epoch 6/100
700/700 [==============================] - 0s 74us/sample - loss: 1.0738 - acc: 0.3657
Epoch 7/100
700/700 [==============================] - 0s 69us/sample - loss: 1.0644 - acc: 0.4286
Epoch 8/100
700/700 [==============================] - 0s 70us/sample - loss: 1.0523 - acc: 0.5586
Epoch 9/100
700/700 [==============================] - 0s 73us/sample - loss: 1.0367 - acc: 0.5400
Epoch 10/100
700/700 [==============================] - 0s 77us/sample - loss: 1.0188 - acc: 0.5971
Epoch 11

Epoch 83/100
700/700 [==============================] - 0s 76us/sample - loss: 0.0419 - acc: 1.0000
Epoch 84/100
700/700 [==============================] - 0s 65us/sample - loss: 0.0406 - acc: 1.0000
Epoch 85/100
700/700 [==============================] - 0s 68us/sample - loss: 0.0393 - acc: 1.0000
Epoch 86/100
700/700 [==============================] - 0s 69us/sample - loss: 0.0379 - acc: 1.0000
Epoch 87/100
700/700 [==============================] - 0s 66us/sample - loss: 0.0368 - acc: 1.0000
Epoch 88/100
700/700 [==============================] - 0s 83us/sample - loss: 0.0359 - acc: 1.0000
Epoch 89/100
700/700 [==============================] - 0s 94us/sample - loss: 0.0347 - acc: 1.0000
Epoch 90/100
700/700 [==============================] - 0s 62us/sample - loss: 0.0336 - acc: 1.0000
Epoch 91/100
700/700 [==============================] - 0s 64us/sample - loss: 0.0325 - acc: 1.0000
Epoch 92/100
700/700 [==============================] - 0s 66us/sample - loss: 0.0315 - acc: 1.0000


In [85]:
# predict
probs = model.predict(X_test)
y_predict = np.argmax(probs, axis=1)
num_errors = np.sum(y_predict != y_test)
print(f'number of misclassifications: {num_errors} out of {len(y_test)} ({100*float(num_errors)/len(y_test):.2f} percent)')

number of misclassifications: 21 out of 300 (7.00 percent)


In [77]:
y_predict[:10]

array([0, 1, 0, 1, 0, 0, 1, 2, 1, 1])

In [78]:
y_test[:10]

array([0, 1, 1, 1, 2, 0, 1, 0, 1, 1], dtype=int32)

In [72]:
df['angle_deg'].min(), df['angle_deg'].max()

(-35.772636, 35.976684999999996)